In [1]:
import tensorflow as tf
import librosa
import numpy as np
import pickle

In [2]:
model = tf.keras.models.load_model('Models//cnn_model_3.h5')

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 128)           768       
                                                                 
 activation (Activation)     (None, 13, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 512)           328192    
                                                                 
 activation_1 (Activation)   (None, 13, 512)           0         
                                                                 
 dropout (Dropout)           (None, 13, 512)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 13, 128)           327808    
                                                                 
 activation_2 (Activation)   (None, 13, 128)           0

In [4]:
def extract_mfcc(signal, sample_rate=22050, n_mfcc=13, n_fft=2048, hop_length=512):
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return np.mean(mfccs.T, axis=0)

In [5]:
with open('Models//scaler_3.pickle', 'rb') as f:
    sc = pickle.load(f)

In [6]:
signal, sample_rate = librosa.load("denoised__non_agg_audio.wav", sr=22050)

features = extract_mfcc(signal)
# Scale data
features = sc.transform(features.reshape(1,-1))
features = np.expand_dims(features, axis=2)

["Aggressive" if model.predict(features)[0][0] > 0.9 else "Non-Aggressive"][0]

1/1 [==============================] - 0s 116ms/step


'Non-Aggressive'

In [7]:
signal, sample_rate = librosa.load("denoised__agg_audio.wav", sr=22050)

features = extract_mfcc(signal)
# Scale data
features = sc.transform(features.reshape(1,-1))
features = np.expand_dims(features, axis=2)

["Aggressive" if model.predict(features)[0][0] > 0.9 else "Non-Aggressive"][0]

1/1 [==============================] - 0s 20ms/step


'Aggressive'